In [69]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import Firefox
from selenium.webdriver.support.ui import WebDriverWait
import os,sys,datetime,random,re
from collections import OrderedDict
import csv
import itertools
import time
from shutil import copyfile,move
import glob
import pandas as pd 
import urllib3
import requests
from clint.textui import progress

In [117]:
STUDY_PATH='/NAS/tupac/protocoles/Strokdem/'
BIDS_PATH='/NAS/deathrow/renaud/dti_disconnection/bids_db/'
myDict = {"romain.viard@chru-lille.fr":"romainviardchru","a.viard@outlook.fr":"aviardoutlook","j.viard@outlook.fr":"jviardoutlook","romain_viard@yahoo.fr":"xIO52ts0UWm5","juliette.mitjans@yahoo.fr":"mdpvolbrain","rhumun@gmail.com":"EYyS9kx1UhWo","renaud.lopes@gmail.com":"Ci2c@chru","clement.bne@gmail.com":"ci2clille","quentinvannodmichel@gmail.com":"p53q37rst","gregkuch@msn.com":"greg2004","e_nedeva@hotmail.com":"VolbrainMdp","jbdavion@hotmail.fr":"neuro2309","qdevignes@gmail.com":"ci2clille"}
myDictSorted=OrderedDict(sorted(myDict.items(), key=lambda t: t[0],reverse=True))

In [71]:
for cle,valeur in myDictSorted.items():
    print(cle,valeur)

romain_viard@yahoo.fr xIO52ts0UWm5
romain.viard@chru-lille.fr romainviardchru
rhumun@gmail.com EYyS9kx1UhWo
renaud.lopes@gmail.com Ci2c@chru
quentinvannodmichel@gmail.com p53q37rst
qdevignes@gmail.com ci2clille
juliette.mitjans@yahoo.fr mdpvolbrain
jbdavion@hotmail.fr neuro2309
j.viard@outlook.fr jviardoutlook
gregkuch@msn.com greg2004
e_nedeva@hotmail.com VolbrainMdp
clement.bne@gmail.com ci2clille
a.viard@outlook.fr aviardoutlook


In [ ]:
def getNbAvailable(idvb,pwd):
    ladate=datetime.datetime.today().strftime('%Y-%m-%d')
    options = Options()
    options.headless = True
    driver = Firefox(options=options)
    driver.implicitly_wait(15)
    #WebDriverWait(driver, timeout=5).until(document_initialised)
    driver.get("http://volbrain.upv.es/index.php")
    elem = driver.find_element_by_name("email")
    elem.send_keys(idvb)
    elem = driver.find_element_by_name("password")
    elem.send_keys(pwd)
    elem = driver.find_element_by_name("sub")
    elem.click()   
    elem = driver.find_element_by_name("job_list")
    html_tab_vol_brain=elem.get_attribute('innerHTML')
    txt_tab_vol_brain=elem.text
    driver.stop_client()       
    driver.close()
    deb=10
    for m in re.finditer(" "+str(ladate)+" ",txt_tab_vol_brain):
        deb -= 1
    return deb

In [ ]:
def sendFileOnVB_volbrain(file,age,sexe,idvb,pwd):
    options = Options()
    options.headless = True
    driver = Firefox(options=options)
    driver.implicitly_wait(15)
   #WebDriverWait(driver, timeout=5).until(document_initialised)
    driver.get("https://volbrain.upv.es/index.php")
    #print("Web site reached")
    elem = driver.find_element_by_name("email")
    elem.send_keys(idvb)
    elem = driver.find_element_by_name("password")
    elem.send_keys(pwd)
    #print("Connected")
    elem = driver.find_element_by_name("sub")
    elem.click()
    elem = driver.find_element_by_id("pipeline1")
    elem.click()
    #print("Pipeline1 Ok")
    elem = driver.find_element_by_name("volbrain_t1_file")
    elem.send_keys(file)
    elem = driver.find_element_by_name("volbrain_patientssex")
    elem.send_keys(sexe)
    elem = driver.find_element_by_name("volbrain_patientsage")
    elem.send_keys(age)
    elem = driver.find_element_by_name("button_volbrain")
    elem.click()
    #print("Data sent")
    driver.stop_client()       
    driver.close()

In [74]:
os.system('rm -rf /tmp/Temp-*')
os.system('rm -rf /tmp/tmpaddon*')
os.system('rm -rf /tmp/rust_mozprofile.*')
os.system('''kill $(ps aux | grep 'firefox'| awk '{print $2}')''')

15

In [73]:
RegNameFile=""
regex="https:\/\/files.volbrain.upv.es.{1,50}?"+str(RegNameFile)+".{1,10}?zip"
idvb="romain.viard@chru-lille.fr"
pwd="romainviardchru"

In [75]:
#def checkIfAlreadySent(RegNameFileList,idvb,pwd):
options = Options()
options.headless = True
driver = Firefox(options=options)
driver.get("http://volbrain.upv.es/index.php")
elem = driver.find_element_by_name("email")
elem.send_keys(idvb)
elem = driver.find_element_by_name("password")
elem.send_keys(pwd)
elem = driver.find_element_by_name("sub")
elem.click()
attempts = 0
cpt=0
time.sleep(10)
elem = driver.find_element_by_name("job_list")
html_tab_vol_brain=elem.get_attribute('innerHTML')
txt_tab_vol_brain=elem.text

for n in range(2,6,1):
    print("page : ",n)
    driver.execute_script("loadJobList("+str(n)+")")
    time.sleep(3)
    elem = driver.find_element_by_name("job_list")
    html_tab_vol_brain=html_tab_vol_brain+elem.get_attribute('innerHTML')
    txt_tab_vol_brain=txt_tab_vol_brain+elem.text

page :  2
page :  3
page :  4
page :  5


In [110]:
del myDictSorted["romain.viard@chru-lille.fr"]
del myDictSorted["rhumun@gmail.com"] 
del myDictSorted["renaud.lopes@gmail.com"] 
del myDictSorted["quentinvannodmichel@gmail.com"] 
del myDictSorted["jbdavion@hotmail.fr"] 
del myDictSorted["qdevignes@gmail.com"] 
del myDictSorted["juliette.mitjans@yahoo.fr"]
del myDictSorted["j.viard@outlook.fr"] 

In [103]:
len(RegNameFileList)

162

In [123]:
del myDictSorted["romain_viard@yahoo.fr"]

for cle,valeur in myDictSorted.items():
    print(cle,valeur)

    os.system('rm -rf /tmp/Temp-*')
    os.system('rm -rf /tmp/tmpaddon*')
    os.system('rm -rf /tmp/rust_mozprofile.*')
    os.system('''kill $(ps aux | grep 'firefox'| awk '{print $2}')''')
    
    #def checkIfAlreadySent(RegNameFileList,idvb,pwd):
    options = Options()
    options.headless = True
    driver = Firefox(options=options)
    driver.get("http://volbrain.upv.es/index.php")
    elem = driver.find_element_by_name("email")
    elem.send_keys(cle)
    elem = driver.find_element_by_name("password")
    elem.send_keys(valeur)
    elem = driver.find_element_by_name("sub")
    elem.click()
    attempts = 0
    cpt=0
    time.sleep(50)
    elem = driver.find_element_by_name("job_list")
    html_tab_vol_brain=elem.get_attribute('innerHTML')
    txt_tab_vol_brain=elem.text

    for n in range(2,5,1):
        print("page : ",n)
        driver.execute_script("loadJobList("+str(n)+")")
        time.sleep(3)
        elem = driver.find_element_by_name("job_list")
        html_tab_vol_brain=html_tab_vol_brain+elem.get_attribute('innerHTML')
        txt_tab_vol_brain=txt_tab_vol_brain+elem.text

    #RegNameFileList=glob.glob(os.path.join(STUDY_PATH,'volbrain','[0-9]*.nii.gz_*.zip'))
    RegNameFileList=glob.glob(os.path.join(STUDY_PATH,'volbrain_wmh_36','native_sub_601224NLL_wmh_tmp.nii.gz_job218892120620150417.zip'))    
    for RegNameFile in RegNameFileList :
        print(RegNameFile)
        RegNameFileBaseName='sub_601224NLL_wmh_tmp.nii.gz_job218892120620150417.zip'#'native_'+os.path.basename(RegNameFile)          
        regex="https:\/\/files.volbrain.upv.es.{1,50}?"+RegNameFileBaseName
        for it in re.finditer(regex,html_tab_vol_brain) :
            mystr=html_tab_vol_brain[it.start():it.end()]
            print(mystr)
            filepath=os.path.join(STUDY_PATH,'volbrain_dl',mystr[30:])
            print(filepath)
            if not os.path.isfile(filepath) :
                attempts = 0
                while attempts < 3:
                    try:
                        r = requests.get(mystr, stream=True)
                        with open(filepath, 'wb') as f:
                            total_length = int(r.headers.get('content-length'))
                            for chunk in progress.bar(r.iter_content(chunk_size=1024), expected_size=(total_length/1024) + 1): 
                                if chunk:
                                    f.write(chunk)
                                    f.flush()
                        break
                    except urllib3.exceptions.NewConnectionError: 
                        attempts += 1
                        print(type(e)) 
    driver.stop_client()
    driver.close()

romain.viard@chru-lille.fr romainviardchru
page :  2
page :  3
page :  4
/NAS/tupac/protocoles/Strokdem/volbrain_wmh_36/native_sub_601224NLL_wmh_tmp.nii.gz_job218892120620150417.zip
https://files.volbrain.upv.es/sub_601224NLL_wmh_tmp.nii.gz_job218892120620150417.zip
/NAS/tupac/protocoles/Strokdem/volbrain_dl/sub_601224NLL_wmh_tmp.nii.gz_job218892120620150417.zip
https://files.volbrain.upv.es/native_sub_601224NLL_wmh_tmp.nii.gz_job218892120620150417.zip
/NAS/tupac/protocoles/Strokdem/volbrain_dl/native_sub_601224NLL_wmh_tmp.nii.gz_job218892120620150417.zip
https://files.volbrain.upv.es/sub_601224NLL_wmh_tmp.nii.gz_job218892120620150417.zip
/NAS/tupac/protocoles/Strokdem/volbrain_dl/sub_601224NLL_wmh_tmp.nii.gz_job218892120620150417.zip
rhumun@gmail.com EYyS9kx1UhWo


NoSuchElementException: Message: Unable to locate element: [name="job_list"]
